In [1]:
import tweepy
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
from nltk.stem.porter import PorterStemmer as PS

In [2]:
config = pd.read_csv("./config.csv")

In [3]:
# Twitter API config
twitterApiKey = config['twitterApiKey'][0]
twitterApiSecret = config['twitterApiSecret'][0]
twitterApiAccessToken = config['twitterApiAccessToken'][0]
twitterApiAccessTokenSecret = config['twitterApiAccessTokenSecret'][0]

In [4]:
# Authenticate
auth = tweepy.OAuthHandler(twitterApiKey, twitterApiSecret)
auth.set_access_token(twitterApiAccessToken, twitterApiAccessTokenSecret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [5]:
# キーワードからツイートを取得
s='btc exclude:retweets'
tweets = tweepy.Cursor(api.search,  
                        count=1,
                        q=s,
                        exclude_replies=True,
                        tweet_mode='extended',
                        show_user=True,
                        lang = 'en',
                        since='2021-07-08',
                        until='2021-07-13',
                        ).items(100);


In [9]:
#ツイートの配列
t_data = []
for tweet in tweets:
    t_data.append([tweet.user.name,'@'+tweet.user.screen_name,tweet.user.friends_count,tweet.user.followers_count,tweet.full_text.replace('\n',''),tweet.favorite_count,tweet.retweet_count])

In [10]:
df = pd.DataFrame(data=t_data, columns=['Name','User Name','Following','Followers','Tweet','Favorite','RT'])
df.head(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT
0,Bitcoin,@BitcoinBlockBot,2,302,"Block #: 690,763Time: 7/12/2021, 11:57:46 PM U...",0,0
1,BitcoinAgile,@bitcoinagile,13903,59378,BTC - we might need to wait a little bit more ...,0,0
2,The Bitcoin Street Journal,@BTCstJournal,1166,10736,BTC – we might need to wait a little bit more ...,1,0


In [11]:
df.tail(10)

,Name,User Name,Following,Followers,Tweet,Favorite,RT
0,Bitcoin,@BitcoinBlockBot,2,302,"Block #: 690,763Time: 7/12/2021, 11:57:46 PM U...",0,0
1,BitcoinAgile,@bitcoinagile,13903,59378,BTC - we might need to wait a little bit more ...,0,0
2,The Bitcoin Street Journal,@BTCstJournal,1166,10736,BTC – we might need to wait a little bit more ...,1,0


In [12]:
#中身の確認
for tweet in tweepy.Cursor(api.search, q='緊急事態宣言 min_faves:400 min_retweets:400 exclude:retweets').items(10):
    print(tweet)


Status(_api=<tweepy.api.API object at 0x00000290F415B310>, _json={'created_at': 'Sat Jul 17 02:38:01 +0000 2021', 'id': 1416225649493168134, 'id_str': '1416225649493168134', 'text': '五輪開催に反対の人がいるは仕方ない思うけど…\n野球やサッカーは観客あり、五輪は観客なし。\n緊急事態宣言で自粛を要請するけど、会長は広島へ行く。\n日本入国後は14日間隔離が通常なのに、3〜6日間でOK。\n飲み会は怒られるけど、歓迎会… https://t.co/hu1cFw8rGF', 'truncated': True, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/hu1cFw8rGF', 'expanded_url': 'https://twitter.com/i/web/status/1416225649493168134', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]}, 'metadata': {'iso_language_code': 'ja', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1019265671287451649, 'id_str': '1019265671287451649', 'name': 'アルトゥル🥰日本

In [13]:
#ステミング
from nltk.stem.porter import PorterStemmer as PS
ps = PS()
ps.stem("I'm looking forward to seeing you ")

"i'm looking forward to seeing you "